# Python для анализа данных

## Веб-скрэйпинг: скачивание файлов

*На основе блокнота Татьяны Рогович, НИУ ВШЭ*
*Автор: Ян Пиле, НИУ ВШЭ*

## Скачивание файлов

Кстати, еще одно применение скрэйпинга, о котором мы пока не поговорили - это автоматизированное скачивание файлов. Например, вы хотите скачать научные статьи по определенному ключевому слову или прайс-листы поставщика, которые он загружает на сайт в эскеле.

Давайте посмотрим, как скачивать файлы на примере pdf и заодно попробуем походить по ссылкам. Кстати, этот процесс еще часто называется spidering или crawling, потому что ваш скрипт как паучок ползет по ссылкам (отсюда и название первых поисковых роботов - spider).

Давайте попробуем сохранить англоязычные summary дисертаций, защищенных в 2019 году

Мы уже отредактировали фильтры и ссылка их запомнила. Позже сегодня посмотрим как можно самим заполнять такие поля с помощью Selenium.

https://www.hse.ru/sci/diss/?author=&chief=&year=2019&type=1&degree_type=&council=&spec=&fulltext=yes

In [2]:
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver as wb
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
driver = wb.Firefox()

driver.implicitly_wait(2)

In [7]:
link = 'https://www.hse.ru/sci/diss/?fulltext=yes'
driver.get(link)

In [6]:
# r = requests.get('https://www.hse.ru/sci/diss/?fulltext=yes')
# r.ok

In [8]:
soup = BeautifulSoup(driver.page_source)
# soup.find_all('a')

In [38]:
# soup.find_all('a', {'class':'link'})

Давайте для начала поэкспериментируем с первым кандидатом.

In [9]:
print(len(soup.find_all('a')))
soup.find_all('a', {'class':'link'})[:10]

177


[<a class="link" href="/sci/diss/641400631">Трансформация репрезентаций украинской национальной идентичности в дискурсах правящих политических элит Украины по вопросу евроатлантической интеграции</a>,
 <a class="link" href="/org/persons/215628947" target="_blank">Жирун Ирина Васильевна</a>,
 <a class="link" href="/org/persons/34164950" target="_blank">Малинова Ольга Юрьевна</a>,
 <a class="link" href="/sci/diss/?keyword=%D0%B4%D0%B8%D1%81%D0%BA%D1%83%D1%80%D1%81">дискурс</a>,
 <a class="link" href="/sci/diss/?keyword=%D0%9D%D0%90%D0%A2%D0%9E">НАТО</a>,
 <a class="link" href="/sci/diss/?keyword=%D0%BD%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F%20%D0%B8%D0%B4%D0%B5%D0%BD%D1%82%D0%B8%D1%87%D0%BD%D0%BE%D1%81%D1%82%D1%8C">национальная идентичность</a>,
 <a class="link" href="/sci/diss/?keyword=%D0%BF%D0%BE%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%D0%B0%20%D0%B8%D0%B4%D0%B5%D0%BD%D1%82%D0%B8%D1%87%D0%BD%D0%BE%D1%81%D1%82%D0%B8">политика идентичности</a>,
 <a class="link" href="/s

Видим, что ссылок очень много, а нам нужны только те, которые ведут на summary. Можно поискать их по тексту ссылки.

In [29]:
links = []
for link in soup.find_all('a', text='Summary'):
    links.append(link.get('href'))

И соберем ссылки в список.

In [11]:
links = []
for link in soup.find_all('a', text='Summary'):
    print(link)
    links.append(link.get('href'))
    
print(links)

<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2022/09/15/1873608165/1Резюме_Жирун ENG 11.08.2022.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2022/08/31/1695860089/Мкртчян_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2022/08/19/1644913549/ИТОГ_БАЛАШОВ_SUMMARY.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2022/10/12/1645724092/Summary Семион.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2022/08/26/1154900249/Миронова_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2022/10/10/1737178694/1Dissertation_introduction.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2022/10/12/1694828628/Ковалёнок_Резюме ENG.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2022/10/06/1663501782/Summary_итог.rtf.pdf">Summary</a>
<a cl

Отлично. Теперь нужно придумать, откуда взять название для каждого файла. Пусть это будут фамилии авторов, давайте доберемся до них. Такую задачу мы пока не решали: будем искать тэг по тексту, а потом искать его родителя (потому что ни ячейку таблицы с именем автора, ни саму таблицу не получится уникально отсечь).

In [35]:
authors = []
for author in soup.find_all('div', {'class':'p1 v', 
                                  'rel':None}):
    try:
        authors.append(author.text)
    except:
        pass


In [41]:
import re

re.match(r'\d+', 'ff16.12.2022') is None

True

In [47]:
authors = [x for x in authors if re.match(r'\d+',x) is None][0::4]

Достанем фамилии.

In [21]:
# authors = []
# for author in soup.find_all('td', text='Соискатель:'):
#     print(author.parent()[1].get_text().split()[0])
#     authors.append(author.parent()[1].get_text().split()[0])
    
# print(authors)

Проверим, что списки действительно одинаковой длины.

In [48]:
len(links)==len(authors)

True

In [32]:
authors

[<a class="link" href="/org/persons/215628947" target="_blank">Жирун Ирина Васильевна</a>,
 <a class="link" href="/org/persons/34164950" target="_blank">Малинова Ольга Юрьевна</a>,
 <a class="link" href="/org/persons/210261427" target="_blank">Балашов Дмитрий Викторович</a>,
 <a class="link" href="/org/persons/68899" target="_blank">Белькович Родион Юрьевич</a>,
 <a class="link" href="/org/persons/65848748" target="_blank">Семион Александр Александрович</a>,
 <a class="link" href="/org/persons/47634799" target="_blank">Афанасьев Валерий Николаевич</a>,
 <a class="link" href="/org/persons/219403371" target="_blank">Смолянская Наталья Владимировна</a>,
 <a class="link" href="/org/persons/10023276" target="_blank">Рыбников Леонид Григорьевич</a>,
 <a class="link" href="/org/persons/46728622" target="_blank">Ковалёнок Анастасия Юрьевна</a>,
 <a class="link" href="/org/persons/61003" target="_blank">Третьяк Ольга Анатольевна</a>,
 <a class="link" href="/org/persons/568698" target="_blank">К

In [49]:
links

['http://www.hse.ru/data/2022/09/15/1873608165/1Резюме_Жирун ENG 11.08.2022.pdf',
 'http://www.hse.ru/data/2022/08/31/1695860089/Мкртчян_summary.pdf',
 'http://www.hse.ru/data/2022/08/19/1644913549/ИТОГ_БАЛАШОВ_SUMMARY.pdf',
 'http://www.hse.ru/data/2022/10/12/1645724092/Summary Семион.pdf',
 'http://www.hse.ru/data/2022/08/26/1154900249/Миронова_summary.pdf',
 'http://www.hse.ru/data/2022/10/10/1737178694/1Dissertation_introduction.pdf',
 'http://www.hse.ru/data/2022/10/12/1694828628/Ковалёнок_Резюме ENG.pdf',
 'http://www.hse.ru/data/2022/10/06/1663501782/Summary_итог.rtf.pdf',
 'http://www.hse.ru/data/2022/10/07/1644047287/Kurakin_summary.pdf',
 'http://www.hse.ru/data/2022/09/09/1690314083/Кузнецова_summary.pdf']

In [51]:
print(links)

['http://www.hse.ru/data/2022/09/15/1873608165/1Резюме_Жирун ENG 11.08.2022.pdf', 'http://www.hse.ru/data/2022/08/31/1695860089/Мкртчян_summary.pdf', 'http://www.hse.ru/data/2022/08/19/1644913549/ИТОГ_БАЛАШОВ_SUMMARY.pdf', 'http://www.hse.ru/data/2022/10/12/1645724092/Summary Семион.pdf', 'http://www.hse.ru/data/2022/08/26/1154900249/Миронова_summary.pdf', 'http://www.hse.ru/data/2022/10/10/1737178694/1Dissertation_introduction.pdf', 'http://www.hse.ru/data/2022/10/12/1694828628/Ковалёнок_Резюме ENG.pdf', 'http://www.hse.ru/data/2022/10/06/1663501782/Summary_итог.rtf.pdf', 'http://www.hse.ru/data/2022/10/07/1644047287/Kurakin_summary.pdf', 'http://www.hse.ru/data/2022/09/09/1690314083/Кузнецова_summary.pdf']


Теперь попробуем сохранить файл. У нас все файлы pdf, будем переименовывать их фамилиями автора. Кстати, если файлы разного формата, то расширение можно узнать через атрибут headers

In [53]:
requests.get(links[0]).headers['content-type']

'application/pdf'

In [54]:
# потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
summary = requests.get(links[0], stream=True) 

# на всякий случай делаем проверку, иначе получим битый файл
if summary.headers['content-type'] == 'application/pdf': 

    # wb - запись байтовой информации
    fh = open('test.pdf', 'wb') 
    
    # считываем туда "содержание" файла по ссылке
    fh.write(summary.content) 
    fh.close()

In [44]:
import os
os.getcwd()

'/Users/i.pile'

Давайте теперь еще добавим имя файла по фамилии.

In [35]:
# потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
summary = requests.get(links[0], stream=True) 

# на всякий случай делаем проверку, иначе получим битый файл
if summary.headers['content-type'] == 'application/pdf': 

    # wb - запись байтовой информации
    fh = open(f'{authors[0]}.pdf', 'wb') 
    
    # считываем туда "содержание" файла по ссылке
    fh.write(summary.content) 
    fh.close()

Давайте сохраним обработку файла в функцию и соберем уже все в цикл.

In [56]:
authors

['Жирун Ирина Васильевна',
 'Мкртчян Арам Арсенович',
 'Балашов Дмитрий Викторович',
 'Семион Александр Александрович',
 'Миронова Татьяна Юрьевна',
 'Машанова-Голикова Инна Антоновна',
 'Ковалёнок Анастасия Юрьевна',
 'Добрушина Екатерина Роландовна',
 'Никулин Александр Михайлович',
 'Осадчий Алексей Евгеньевич']

In [57]:
def get_pdf(idx):
    summary = requests.get(links[idx], stream=True) # потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
    if summary.headers['content-type'] == 'application/pdf': # на всякий случай делаем проверку, иначе получим битый файл
        s = authors[idx]
        fh = open(f'{s}.pdf', 'wb') # wb - запись байтовой информации
        fh.write(summary.content) # считываем туда "содержание" файла по ссылке
        fh.close()

In [71]:
import re
s = re.findall(r'\w+',links[0].split('/')[-1])[0]

In [72]:
f'{s}.pdf'

'Немова.pdf'

In [41]:
link = 'https://www.hse.ru/sci/diss/?fulltext=yes'
driver.get(link)

soup = BeautifulSoup(driver.page_source)

links = []
for link in soup.find_all('a', text='Summary'):
    links.append(link.get('href'))

authors = []
for author in soup.find_all('td', text='Соискатель:'):
    authors.append(author.parent()[1].get_text().split()[0])

In [58]:
links


['http://www.hse.ru/data/2022/09/15/1873608165/1Резюме_Жирун ENG 11.08.2022.pdf',
 'http://www.hse.ru/data/2022/08/31/1695860089/Мкртчян_summary.pdf',
 'http://www.hse.ru/data/2022/08/19/1644913549/ИТОГ_БАЛАШОВ_SUMMARY.pdf',
 'http://www.hse.ru/data/2022/10/12/1645724092/Summary Семион.pdf',
 'http://www.hse.ru/data/2022/08/26/1154900249/Миронова_summary.pdf',
 'http://www.hse.ru/data/2022/10/10/1737178694/1Dissertation_introduction.pdf',
 'http://www.hse.ru/data/2022/10/12/1694828628/Ковалёнок_Резюме ENG.pdf',
 'http://www.hse.ru/data/2022/10/06/1663501782/Summary_итог.rtf.pdf',
 'http://www.hse.ru/data/2022/10/07/1644047287/Kurakin_summary.pdf',
 'http://www.hse.ru/data/2022/09/09/1690314083/Кузнецова_summary.pdf']

In [59]:
for idx in range(len(links)):
    get_pdf(idx)

Готово!